<a href="https://colab.research.google.com/github/flavio641/Modelos_Aprendizados_Maquinas/blob/main/VariaveisImportantes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('/content/feat_imp.csv')
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,Exited
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,1,0,1
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,1,0,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,1,0,1
3,699,39,1,0.00,2,0,0,93826.63,1,0,0,1,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,1,0,0,0,1,0
9996,516,35,10,57369.61,1,1,1,101699.77,1,0,0,0,1,0
9997,709,36,7,0.00,1,0,1,42085.58,1,0,0,1,0,1
9998,772,42,3,75075.31,2,1,0,92888.52,0,1,0,0,1,1


# ***Separação da variáveis preditivas e variável resposta***

In [3]:
X = df.drop('Exited', axis= 1)
y = df.Exited

In [4]:
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(X,y, test_size = 0.3, random_state = 61658)

# ***Definição dos Parâmetros Validação Cruzada***

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {
    'max_leaf_nodes': [4,8,16,32,64,128,256,512,2014],
    'criterion': ['gini', 'entropy']
}

grid = GridSearchCV(
    DecisionTreeClassifier(random_state= 61658),
    params,
    cv= 10,
    scoring = 'roc_auc',
    verbose = 5
)

In [10]:
grid.fit(x_treino, y_treino)

Fitting 10 folds for each of 18 candidates, totalling 180 fits
[CV 1/10] END .criterion=gini, max_leaf_nodes=4;, score=0.756 total time=   0.0s
[CV 2/10] END .criterion=gini, max_leaf_nodes=4;, score=0.731 total time=   0.0s
[CV 3/10] END .criterion=gini, max_leaf_nodes=4;, score=0.756 total time=   0.0s
[CV 4/10] END .criterion=gini, max_leaf_nodes=4;, score=0.757 total time=   0.0s
[CV 5/10] END .criterion=gini, max_leaf_nodes=4;, score=0.762 total time=   0.0s
[CV 6/10] END .criterion=gini, max_leaf_nodes=4;, score=0.738 total time=   0.0s
[CV 7/10] END .criterion=gini, max_leaf_nodes=4;, score=0.715 total time=   0.0s
[CV 8/10] END .criterion=gini, max_leaf_nodes=4;, score=0.726 total time=   0.0s
[CV 9/10] END .criterion=gini, max_leaf_nodes=4;, score=0.730 total time=   0.0s
[CV 10/10] END criterion=gini, max_leaf_nodes=4;, score=0.756 total time=   0.0s
[CV 1/10] END .criterion=gini, max_leaf_nodes=8;, score=0.818 total time=   0.0s
[CV 2/10] END .criterion=gini, max_leaf_nodes=

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=61658),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_leaf_nodes': [4, 8, 16, 32, 64, 128, 256, 512,
                                            2014]},
             scoring='roc_auc', verbose=5)

# ***Qual foi o melhor parâmetro do modelo***

In [11]:
grid.best_params_

{'criterion': 'entropy', 'max_leaf_nodes': 32}

In [12]:
grid.best_score_  

0.844584741625602

In [13]:
y_pred = grid.predict_proba(x_teste)
y_pred

array([[0.97933001, 0.02066999],
       [0.93684211, 0.06315789],
       [0.92800789, 0.07199211],
       ...,
       [0.79723502, 0.20276498],
       [0.97933001, 0.02066999],
       [0.97933001, 0.02066999]])

In [17]:
y_pred = y_pred[:,1]

In [18]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_teste, y_pred)


0.8461416666666665

# ***Verificar a Importância das Variáveis***

In [19]:
grid.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_leaf_nodes=32,
                       random_state=61658)

In [20]:
grid.best_estimator_.feature_importances_

array([0.        , 0.39756168, 0.        , 0.12983555, 0.30706264,
       0.        , 0.11164571, 0.00618282, 0.        , 0.04771159,
       0.        , 0.        , 0.        ])

In [55]:
importancias = grid.best_estimator_.feature_importances_
ordem = np.argsort(importancias)[::-1]
colunas = X.columns

for col, imp, in zip(colunas[ordem], importancias[ordem]):
  print(col, imp)

Age 0.39756167881330673
NumOfProducts 0.3070626438661241
Balance 0.1298355548570011
IsActiveMember 0.11164570613828362
Geography_Germany 0.04771159444814778
EstimatedSalary 0.006182821877136774
Gender_Male 0.0
Gender_Female 0.0
Geography_Spain 0.0
Geography_France 0.0
HasCrCard 0.0
Tenure 0.0
CreditScore 0.0


In [61]:
colunas_selecionadas = [
    'Age',
    'Balance',
    'NumOfProducts',
    'IsActiveMember',
    'Geography_Germany',
    'EstimatedSalary',
]
grid.fit(x_treino[colunas_selecionadas], y_treino)

Fitting 10 folds for each of 18 candidates, totalling 180 fits
[CV 1/10] END .criterion=gini, max_leaf_nodes=4;, score=0.756 total time=   0.0s
[CV 2/10] END .criterion=gini, max_leaf_nodes=4;, score=0.731 total time=   0.0s
[CV 3/10] END .criterion=gini, max_leaf_nodes=4;, score=0.756 total time=   0.0s
[CV 4/10] END .criterion=gini, max_leaf_nodes=4;, score=0.757 total time=   0.0s
[CV 5/10] END .criterion=gini, max_leaf_nodes=4;, score=0.762 total time=   0.0s
[CV 6/10] END .criterion=gini, max_leaf_nodes=4;, score=0.738 total time=   0.0s
[CV 7/10] END .criterion=gini, max_leaf_nodes=4;, score=0.715 total time=   0.0s
[CV 8/10] END .criterion=gini, max_leaf_nodes=4;, score=0.726 total time=   0.0s
[CV 9/10] END .criterion=gini, max_leaf_nodes=4;, score=0.730 total time=   0.0s
[CV 10/10] END criterion=gini, max_leaf_nodes=4;, score=0.756 total time=   0.0s
[CV 1/10] END .criterion=gini, max_leaf_nodes=8;, score=0.818 total time=   0.0s
[CV 2/10] END .criterion=gini, max_leaf_nodes=

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=61658),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_leaf_nodes': [4, 8, 16, 32, 64, 128, 256, 512,
                                            2014]},
             scoring='roc_auc', verbose=5)

In [62]:
y_pred2 = grid.predict_proba(x_teste[colunas_selecionadas])[:,1]
roc_auc_score(y_teste, y_pred2)

0.8461416666666665